## Exercise 1: UDP Chat Application

This exercise uses UDP to build a simple chat application. Clients send messages to a central server that broadcasts them to all other clients.

In [2]:
import socket

def run_udp_chat_server(host='localhost', port=65433):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    server_socket.bind((host, port))
    print(f"UDP Chat Server is running on {host}:{port}...")
    
    # Dictionary to map client addresses to usernames
    clients = {}
    
    try:
        while True:
            data, client_address = server_socket.recvfrom(2048)
            message = data.decode().strip()
            
            # If new client, treat first message as username
            if client_address not in clients:
                username = message
                clients[client_address] = username
                print(f"New client: {client_address} as {username}")
                server_socket.sendto(f"Welcome, {username}!".encode(), client_address)
            else:
                sender = clients[client_address]
                broadcast_msg = f"{sender}: {message}"
                print(f"Broadcasting: {broadcast_msg}")
                for addr in clients:
                    if addr != client_address:
                        server_socket.sendto(broadcast_msg.encode(), addr)
    except KeyboardInterrupt:
        print("\nShutting down chat server.")
    finally:
        server_socket.close()

# Uncomment the next line to run the chat server
# run_udp_chat_server()

### UDP Chat Client

Below is a simple UDP chat client. Run this in a separate terminal or notebook cell. The first message sent is considered the username.

In [3]:
import socket
import threading

def receive_messages(client_socket):
    while True:
        try:
            data, _ = client_socket.recvfrom(2048)
            print("\n" + data.decode())
        except Exception as e:
            print("Error receiving:", e)
            break

def run_udp_chat_client(server_host='localhost', server_port=65433):
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    server_address = (server_host, server_port)
    
    # Start a thread to receive messages
    thread = threading.Thread(target=receive_messages, args=(client_socket,), daemon=True)
    thread.start()
    
    print("UDP Chat Client. Type your messages below. Your first message will be your username.")
    try:
        while True:
            msg = input()
            if msg:
                client_socket.sendto(msg.encode(), server_address)
    except KeyboardInterrupt:
        print("Exiting chat client.")
    finally:
        client_socket.close()

# Uncomment the next line to run the chat client
# run_udp_chat_client()

## Exercise 2: Mapping User IP Addresses

In this exercise, we create a simple dictionary that maps a client's IP address to a username. This example is embedded in the chat server above. The `clients` dictionary in the server maps each client address to a username.

You can view the dictionary by printing it on the server side.

## Exercise 3: Adding Authentication

Enhance the UDP chat server so that clients must authenticate before chatting. In this simple example, the server expects the first message from a client to be in the format `username:password`. Only if the password matches a preset value (e.g. `pass123`), the client is accepted.

In [5]:
import socket

def run_udp_auth_server(host='localhost', port=65433, valid_password='pass123'):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    server_socket.bind((host, port))
    print(f"UDP Auth Server is running on {host}:{port}...")
    
    # Dictionary to store authenticated clients
    clients = {}
    
    try:
        while True:
            data, client_address = server_socket.recvfrom(2048)
            message = data.decode().strip()
            
            # If client is not authenticated, expect username:password
            if client_address not in clients:
                if ':' in message:
                    username, password = message.split(':', 1)
                    if password == valid_password:
                        clients[client_address] = username
                        server_socket.sendto(f"Welcome, {username}!".encode(), client_address)
                        print(f"Authenticated {client_address} as {username}")
                    else:
                        server_socket.sendto("Authentication failed.".encode(), client_address)
                else:
                    server_socket.sendto("Please send credentials as username:password".encode(), client_address)
            else:
                sender = clients[client_address]
                broadcast_msg = f"{sender}: {message}"
                print(f"Broadcasting: {broadcast_msg}")
                for addr in clients:
                    if addr != client_address:
                        server_socket.sendto(broadcast_msg.encode(), addr)
    except KeyboardInterrupt:
        print("\nShutting down auth server.")
    finally:
        server_socket.close()

# Uncomment the next line to run the authentication server
# run_udp_auth_server()

## Exercise 4: Encrypting and Decrypting Messages

In this exercise, we use the Fernet module from the `cryptography` library to encrypt messages on the client side and decrypt them on the server side. (Make sure to install the cryptography package: `pip install cryptography`.)

In [7]:
from cryptography.fernet import Fernet

# Generate a key (this should be done once and shared securely between client and server)
key = Fernet.generate_key()
cipher = Fernet(key)

def encrypt_message(message):
    return cipher.encrypt(message.encode())

def decrypt_message(token):
    return cipher.decrypt(token).decode()

# Example usage:
original = "Hello, secure UDP!"
encrypted = encrypt_message(original)
decrypted = decrypt_message(encrypted)
print("Original:", original)
print("Encrypted:", encrypted)
print("Decrypted:", decrypted)

Original: Hello, secure UDP!
Encrypted: b'gAAAAABn2wDbKKo_F98vGi46Ht45d4Dp-hojwznp0b6qLCm6VzcK7Gkz1mpeNkRlCQTmtfbe3rcVeTjDKzz_HhC_MOSXT7G4hXN2A_DuGbjHk3UJ5WE6Wlo='
Decrypted: Hello, secure UDP!


## Exercise 5: API Data Collection Application

This exercise demonstrates fetching weather data from an API (using Open-Meteo) and sending it via UDP to a server.

In [9]:
import socket
import requests

# Fetch weather data from Open-Meteo API
api_url = "https://api.open-meteo.com/v1/forecast?latitude=51.47&longitude=-0.0363&current_weather=true"
response = requests.get(api_url)

if response.status_code == 200:
    weather_data = response.json()
    temperature = weather_data["current_weather"]["temperature"]
    message = f"Current temperature: {temperature}°C"
else:
    message = "Failed to fetch weather data"

# Send the weather data using UDP
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ('localhost', 65433)  
client_socket.sendto(message.encode(), server_address)
print("Weather data sent!")
client_socket.close()

Weather data sent!


## Exercise 6: Compare Temperature Between Two Locations

In this exercise, update the API data collection script to fetch weather data for two locations (for example, one for your university and one for the British Library) and compare the temperatures.

In [10]:
import requests

def get_temperature(latitude, longitude):
    api_url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
    response = requests.get(api_url)
    if response.status_code == 200:
        weather_data = response.json()
        return weather_data["current_weather"]["temperature"]
    else:
        return None

# Example coordinates (replace with actual coordinates):
# University (e.g., 51.5, -0.1) and British Library (e.g., 51.52, -0.12)
uni_lat, uni_lon = 51.5, -0.1
brit_lat, brit_lon = 51.52, -0.12

temp_uni = get_temperature(uni_lat, uni_lon)
temp_brit = get_temperature(brit_lat, brit_lon)

if temp_uni is not None and temp_brit is not None:
    print(f"University temperature: {temp_uni}°C")
    print(f"British Library temperature: {temp_brit}°C")
    diff = abs(temp_uni - temp_brit)
    print(f"Temperature difference: {diff}°C")
else:
    print("Failed to fetch temperature data for one or both locations.")

University temperature: 15.9°C
British Library temperature: 15.9°C
Temperature difference: 0.0°C
